In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# --- Load Data ---
df = pd.read_csv('../data/processed/train_merged.csv', low_memory=False, parse_dates=['Date'])

# --- Feature Engineering ---
# Date features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

# Competition features
df['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
df['CompetitionOpenSinceYear'].fillna(0, inplace=True)
df['CompetitionOpen'] = (df['Year'] - df['CompetitionOpenSinceYear']) * 12 + \
                        (df['Month'] - df['CompetitionOpenSinceMonth'])
df['CompetitionOpen'] = df['CompetitionOpen'].apply(lambda x: max(x, 0))
df['CompetitionDistance'].fillna(df['CompetitionDistance'].median(), inplace=True)

# Promo2 features
df['Promo2SinceWeek'].fillna(0, inplace=True)
df['Promo2SinceYear'].fillna(0, inplace=True)
df['PromoInterval'].fillna('', inplace=True)

def is_promo2_active(row):
    if row['Promo2'] == 0:
        return 0
    promo2_start_year = int(row['Promo2SinceYear'])
    promo2_start_week = int(row['Promo2SinceWeek'])
    current_year = row['Year']
    current_week = row['WeekOfYear']
    if current_year < promo2_start_year:
        return 0
    if current_year == promo2_start_year and current_week < promo2_start_week:
        return 0
    month_str = row['Date'].strftime('%b')
    if month_str in row['PromoInterval']:
        return 1
    else:
        return 0

df['IsPromo2'] = df.apply(is_promo2_active, axis=1)

print('Data loaded and features engineered.')
df.head()

### 2. Feature Selection ve Categorical Encoding

In [ ]:
# Kategorik ve numerik feature'ları seçelim
features = [
    # Zaman
    'Year', 'Month', 'Day', 'DayOfWeek', 'WeekOfYear',
    # Mağaza
    'Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpen',
    # Promosyon
    'Promo', 'Promo2', 'IsPromo2',
    # Diğer
    'StateHoliday', 'SchoolHoliday'
]

target = 'Sales'

# Kategorik olanları encode edelim
categorical_features = ['StoreType', 'Assortment', 'StateHoliday']
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])

print('Features selected and encoded.')

### 3. Eğitim ve Validasyon Seti Oluşturma

In [ ]:
# Sadece mağazalar açıkken ve satış varken olan veriyi alalım
df_train = df[(df['Open'] == 1) & (df['Sales'] > 0)]

validation_date = df_train['Date'].max() - pd.DateOffset(weeks=6)
train_indices = df_train['Date'] < validation_date
val_indices = df_train['Date'] >= validation_date

X_train, y_train = df_train[train_indices][features], df_train[train_indices][target]
X_val, y_val = df_train[val_indices][features], df_train[val_indices][target]

print('Training set shape:', X_train.shape)
print('Validation set shape:', X_val.shape)

### 4. XGBoost Modelini Eğitme

In [ ]:
def rmsp_error_xgb(y_pred, y_true):
    y_true = y_true.get_label()
    # Satışların 0 olduğu durumları hesaba katma
    # XGBoost DMatrix'i 0'a bölme hatası vermemesi için küçük bir epsilon ekleyebiliriz.
    y_true[y_true == 0] = 1e-6
    
    percentage_error = (y_true - y_pred) / y_true
    rmspe = np.sqrt(np.mean(np.square(percentage_error)))
    return 'rmspe', rmspe

params = {
    'objective': 'reg:squarederror', # Regresyon görevi
    'eta': 0.05,                     # Learning rate
    'max_depth': 8,                  # Ağaç derinliği
    'subsample': 0.7,                # Her ağaç için kullanılacak veri oranı
    'colsample_bytree': 0.7,         # Her ağaç için kullanılacak özellik oranı
    'eval_metric': 'rmse',           # Değerlendirme metriği (kendi fonksiyonumuzu da kullanacağız)
    'seed': 42                       # Tekrarlanabilirlik için
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

watchlist = [(dtrain, 'train'), (dval, 'eval')]

model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000, # Yüksek bir değer, early stopping ile duracak
    evals=watchlist,
    custom_metric=rmsp_error_xgb,
    maximize=False, # RMSPE'yi minimize etmeye çalışıyoruz
    early_stopping_rounds=50, # 50 turda iyileşme olmazsa dur
    verbose_eval=100
)

print('Model trained.')

### 5. Model Değerlendirme

In [ ]:
def rmsp_error(y_true, y_pred):
    y_true = y_true.to_numpy()
    y_pred = y_pred.to_numpy()
    # Satışların 0 olduğu durumları hesaba katma
    y_true = y_true[y_pred != 0]
    y_pred = y_pred[y_pred != 0]
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

y_pred = model.predict(dval)
val_rmspe = rmsp_error(y_val, pd.Series(y_pred, index=y_val.index))

print(f'Validation RMSPE: {val_rmspe:.4f}')